In [26]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

import pandas as pd
import os
import copy

def pt(d, t):
    if d:
        Tree.fromstring(str(t)).pretty_print()

In [23]:
ts0 = "(NTA (PA a) (NTB (NTC (NTD (PB b) (NTE (PC c) (NTF (NTG (PD d) (PE e)) (PF f)))) (PG g)) (PH h)))"
ts1 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts2 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
ts3 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
ts4 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts5 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
ts6 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
ts7 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
ts8 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
ts9 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"

## Encoding playground

In [28]:
all_ts = [ts0, ts1, ts2, ts3, ts4, ts5, ts6, ts7, ts8, ts9]
debug = False
encoder = C_NaiveAbsoluteEncoding(separator="_",
                                    unary_joiner="++",
                                    reverse=True,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="[*]")
for ts in all_ts:
    pt(debug, ts)
    
    tree = C_Tree.from_string(ts)
    lc_tree = encoder.encode(tree)
    dec_tree = encoder.decode(lc_tree)
    dec_tree = dec_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    pt(debug, dec_tree)
    
    assert str(dec_tree) == ts

## Labels equivalence

In [2]:
ptb_path = "~/Treebanks/const/PENN_TREEBANK/"
ptb_path = os.path.expanduser(ptb_path)
print("Reading PTB from",ptb_path)    
df = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
with open(os.path.join(ptb_path,"test.trees")) as f:
    ptb = f.read().splitlines()

for tree_string in ptb:
    tree = C_Tree.from_string(tree_string)
    rel_encoder = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=False, binary=True, binary_marker="[*]")
    tet_encoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False, binary_marker="[*]")

    t_lin_tree = tet_encoder.encode(copy.deepcopy(tree))
    r_lin_tree = rel_encoder.encode(copy.deepcopy(tree))

    df_temp = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
    
    df_temp["Word"] = r_lin_tree.words
    df_temp["Relative"] = [str(l.n_commons) for l in r_lin_tree.labels]
    df_temp["Tetratag"] = [l.n_commons for l in t_lin_tree.labels]
    
    df = pd.concat([df, df_temp], axis=0)

    
    r_decoded_tree = rel_encoder.decode(r_lin_tree)
    r_decoded_tree = r_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    t_decoded_tree = tet_encoder.decode(t_lin_tree)
    t_decoded_tree = t_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
        
    if not (str(r_decoded_tree)==tree_string):
        print("Decoded tree is different from original")
        print("Original:",tree_string)
        print("Decoded:",r_decoded_tree)

Reading PTB from /home/droca1/Treebanks/const/PENN_TREEBANK/


In [3]:
# print relative values for each of the unique values of tetratag
tetra_values = df["Tetratag"].unique()
tetra_values.sort()
for tetratag in tetra_values:
    print("Tetratag:",tetratag,\
        "\n\t==> Relative:",df[df["Tetratag"]==tetratag]["Relative"].unique())

Tetratag: l 
	==> Relative: ['-3' '-4' '-1' '-2' '-5' '0' '-6' '-7' '-12' '-9' '-14' '-8' '-10' '-11']
Tetratag: lL 
	==> Relative: ['-3' '-4' '-1' '-5' '-13' '-6' '-2' '-11' '-8' '-14' '-7' '-10' '-12'
 '-9' '-19' '-16' '-15' '-17' '-27' '-22' '-20' '-18' '-25' '-21' '-23'
 '-30' '-26']
Tetratag: lR 
	==> Relative: ['-4' '-6' '-5' '-3' '-2' '-13' '-1' '-15' '-7' '-9' '-11' '-8' '-17'
 '-10' '-14' '-12' '-16' '-19']
Tetratag: r 
	==> Relative: ['1']
Tetratag: rL 
	==> Relative: ['1' '2' '0' '3']
Tetratag: rR 
	==> Relative: ['1' '2' '3' '4' '5' '6']
